In [46]:
import pandas as pd
import altair as alt
alt.data_transformers.enable('vegafusion')
import networkx as nx

### Question

**Where are differences in throughput times between the municipalities and how can these be explained?**

In [47]:
df = pd.read_csv('bpic2015_labeled.csv')
print(len(df))
df.groupby('case:parts').size().to_frame().sort_values(by=0, ascending=False)

25073


,0
case:parts,
Kap,25073


In [48]:
df = df[df['case:parts'] == 'Kap']
print(df['subprocess'].unique())
print(df.columns)

# remove extreme outliers based on case duration (result: only one case in muni-5 is removed)
# print(df[df['case_duration_days'] > 400])
df = df[df['case_duration_days'] <= 400]

df

['01_HOOFD' '01_BB']
Index(['Unnamed: 0.1', 'key_0', 'Unnamed: 0', 'case:Responsible_actor',
       'case:SUMleges', 'case:last_phase', 'case:parts', 'case:concept:name',
       'concept:name', 'lifecycle:transition', 'municipality', 'org:resource',
       'time:timestamp', 'case:parts_Bouw', 'subprocess', 'phase', '@@index',
       '@@case_index', 'date', 'activity_duration_seconds',
       'activity_duration_days', 'case_duration_seconds', 'case_duration_days',
       'case_type', 'activity_type', 'time_type'],
      dtype='object')


,Unnamed: 0.1,key_0,Unnamed: 0,case:Responsible_actor,case:SUMleges,case:last_phase,case:parts,case:concept:name,concept:name,lifecycle:transition,...,@@index,@@case_index,date,activity_duration_seconds,activity_duration_days,case_duration_seconds,case_duration_days,case_type,activity_type,time_type
0,0,65523,42,560596.0,16.86000,Vergunning onherroepelijk,Kap,3398124,01_HOOFD_010,complete,...,65523,1409,2010-10-10,NaN,NaN,9897780.0,114.557639,CT.13,AT.2,TT.1
1,1,65524,83,560596.0,16.86000,Vergunning onherroepelijk,Kap,3398124,01_HOOFD_015,complete,...,65524,1409,2010-10-11,135480.0,1.568056,9897780.0,114.557639,CT.13,AT.2,TT.0
2,2,65525,84,560596.0,16.86000,Vergunning onherroepelijk,Kap,3398124,01_HOOFD_020,complete,...,65525,1409,2010-10-11,0.0,0.000000,9897780.0,114.557639,CT.13,AT.2,TT.0
3,3,53156,86,560872.0,NaN,Vergunning verleend,Kap,2797217,01_HOOFD_030_2,complete,...,53156,1085,2010-10-12,NaN,NaN,8440680.0,97.693056,CT.16,AT.2,TT.5
4,4,53157,87,560872.0,NaN,Vergunning verleend,Kap,2797217,01_HOOFD_010,complete,...,53157,1085,2010-10-12,0.0,0.000000,8440680.0,97.693056,CT.16,AT.2,TT.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25068,25068,52714,262465,560519.0,70.81200,Beschikking gereed,Kap,23867068,01_HOOFD_250_2,complete,...,52714,1064,2015-03-04,0.0,0.000000,3715200.0,43.000000,CT.11,AT.2,TT.3
25069,25069,52726,262590,560519.0,70.81200,Beschikking gereed,Kap,23867068,01_HOOFD_490_1a,complete,...,52726,1064,2015-03-04,0.0,0.000000,3715200.0,43.000000,CT.11,AT.2,TT.3
25070,25070,241397,262602,560696.0,NaN,Aanvraag ontvangen,Kap,8484936,01_HOOFD_030_2,complete,...,241397,5247,2015-03-05,55680.0,0.644444,1178880.0,13.644444,CT.4,AT.2,TT.5
25071,25071,24418,262626,560872.0,220.82385,Beschikking verzonden,Kap,12999831,01_HOOFD_510_2,complete,...,24418,541,2015-03-09,518400.0,6.000000,2419200.0,28.000000,CT.16,AT.2,TT.0


### Report on the preprocessed dataset stats

In [49]:
print(df['case:concept:name'].nunique())
print(len(df))
print(df['concept:name'].nunique())
print(df['org:resource'].nunique())

667
25026
129
38


### Result (related to Kap analysis only)

- We select only the cases with 'case:part' attribute having value 'Kap'
- This means 'cutting down trees' in the context of the BPIC15 dataset
- Cases in this group is the 2nd most frequent cases across municipalities
- The selection of cases is a priori -- we wish to use a variant of cases for which some of the municipalities share similarities in performance (in terms of case duration time) while the rest differ

In [50]:
# map types to their actual names
# extracted from 'SASearchMiner-bpic15_20231120-135246.881940.miner'
# ct_map = {
    # 'CT.0': {'Bezwaar of beroep ontvangen'}, 
    # 'CT.1': {'Procedure hervat'}, 
    # 'CT.2': {'Aanvraag ontvangen'}, 
    # 'CT.3': {'Procedure tussentijds beëindigd'}, 
    # 'CT.4': {'Besluit onherroepelijk', 'Doorgezonden aan bevoegd gezag'}, 
    # 'CT.5': {'Aanvullende gegevens gevraagd', 'Beschikking verzonden', 'Ontwerpbesluit genomen', 'Vergunning verleend'}, 
    # 'CT.6': {'Beschikking gereed', 'Zaak afgehandeld'}, 
    # 'CT.7': {'Besluit genomen'}, 
    # 'CT.8': {'Procedure afgebroken'}, 
    # 'CT.9': {'Activiteit vergunningvrij'}, 
    # 'CT.10': {'Vergunning onherroepelijk'}, 
    # 'CT.11': {'Bezwaar ingediend'}, 
    # 'CT.12': {'Advies bekend'}
#     'CT.0': {'Objection or appeal received'},
#     'CT.1': {'Procedure resumed'},
#     'CT.2': {'Application received'},
#     'CT.3': {'Procedure terminated interim'},
#     'CT.4': {'Decision irrevocable', 'Forwarded to competent authority'},
#     'CT.5': {'Additional information requested', 'Decision sent', 'Draft decision made', 'Permit granted'},
#     'CT.6': {'Decision ready', 'Case handled'},
#     'CT.7': {'Decision made'},
#     'CT.8': {'Procedure aborted'},
#     'CT.9': {'Activity exempt from permit'},
#     'CT.10': {'Permit irrevocable'},
#     'CT.11': {'Objection filed'},
#     'CT.12': {'Advice known'}
# }

# extracted from 'SASearchMiner-bpic15_20231120-160154.502757.miner'
ct_map = {
    'CT.0': {'Objection or appeal received'},
    'CT.1': {'Procedure resumed'},
    'CT.2': {'Decision irrevocable'},
    'CT.3': {'Case handled'},
    'CT.4': {'Application received'},
    'CT.5': {'Procedure terminated interim'},
    'CT.6': {'Decision made'},
    'CT.7': {'Draft decision made'},
    'CT.8': {'Procedure aborted'},
    'CT.9': {'Activity exempt from permit'},
    'CT.10': {'Additional information requested'},
    'CT.11': {'Decision ready'},
    'CT.12': {'Forwarded to competent authority'},
    'CT.13': {'Permit irrevocable'},
    'CT.14': {'Objection filed'},
    'CT.15': {'Advice known'},
    'CT.16': {'Decision sent', 'Permit granted'}
}
at_map = {
    # 'AT.0': 'phase [01_BB_5, 01_BB_7]', 
    'AT.0': 'objections phase 5 & 7', 
    # 'AT.1': 'phase [01_BB_6]', 
    'AT.1': 'objections phase 6',
    # 'AT.2': 'phase [01_HOOFD_0-8, except 7]', 
    'AT.2': 'main phase 0-6 & 8',
    # 'AT.3': 'phase [01_HOOFD_7]',
    'AT.3': 'main phase 7',
}
tt_map = {
    # 'TT.0': 'Monday',
    'TT.0': 'TT.1 (Mon)',
    # 'TT.1': 'Sunday',
    'TT.1': 'TT.7 (Sun)',
    # 'TT.2': 'Friday', 
    'TT.2': 'TT.5 (Fri)',
    # 'TT.3': 'Wednesday', 
    'TT.3': 'TT.3 (Wed)',
    # 'TT.4': 'Saturday', 
    'TT.4': 'TT.6 (Sat)',
    # 'TT.5': 'Tuesday & Thursday',
    'TT.5': 'TT.2 (Tue & Thu)',
}

df['case_type'] = df['case_type'].apply(
    lambda x: '{} ({})'.format(x, '/'.join(sorted(ct_map[x])))
)
df['activity_type'] = df['activity_type'].apply(
    lambda x: '{} ({})'.format(x, at_map[x])
)
df['time_type'] = df['time_type'].apply(
    lambda x: '{}'.format(tt_map[x])
)

## Case-oriented analysis

In [51]:
# derive case log

def reassign_case_level_attr_val(arr):
    # arr records values for a case level attribute
    if arr.nunique() > 1:
        raise ValueError('Not a case-level attribute')
    else:
        return arr.unique()[0]

df_gb = df.groupby('case:concept:name').agg(
    case_duration_days=pd.NamedAgg(column='case_duration_days', aggfunc='mean'),
    case_type=pd.NamedAgg(column='case_type', aggfunc=reassign_case_level_attr_val),
    # case_Responsible_actor=pd.NamedAgg(column='case:Responsible_actor', aggfunc=reassign_case_level_attr_val),
    muni=pd.NamedAgg(column='municipality', aggfunc=reassign_case_level_attr_val),
    num_events=pd.NamedAgg(column='case_duration_days', aggfunc='count')
)
df_gb = df_gb.reset_index()
df_gb

,case:concept:name,case_duration_days,case_type,muni,num_events
0,2797217,97.693056,CT.16 (Decision sent/Permit granted),muni-1,40
1,2814003,83.677778,CT.16 (Decision sent/Permit granted),muni-1,38
2,2839858,80.381250,CT.15 (Advice known),muni-1,38
3,2840219,80.384028,CT.15 (Advice known),muni-1,38
4,2840553,80.376389,CT.15 (Advice known),muni-1,38
...,...,...,...,...,...
662,23775864,42.000000,CT.16 (Decision sent/Permit granted),muni-2,37
663,23860509,53.402083,CT.16 (Decision sent/Permit granted),muni-2,37
664,23865607,42.414583,CT.16 (Decision sent/Permit granted),muni-2,38
665,23867068,43.000000,CT.11 (Decision ready),muni-2,30


In [52]:
# Test if there are differences in case duration across municipalities
# Kruskal-Wallis H-Test
# Ranks-based test (cf. Wilcoxon rank sum test)
# Condition 1: k samples being random and independent (yes; assuming five municipalities are independent)
# Condition 2: all k samples having five or more measurements (yes; min sample size > 800)
# Condition 3: the k prob. distributions are continuous (yes; case duration time)
# H0: The 5 distributions (one for each municipality) of case duration time are identical
# Ha: At least two of the 5 distributions differ in location
# Rejection region: H > Chi-Square with (5-1) df.
from scipy.stats import kruskal

case_dur_munis = [df_gb.loc[df_gb['muni'] == muni, 'case_duration_days'] for muni in sorted(df_gb['muni'].unique())]
# print(case_dur_munis)
# H (i.e., Chi-Square) and p-value
print(kruskal(*case_dur_munis))
# Result: p-value << 0.01
# i.e., at alpha = 1% level, the differences are significant

# viz using boxplot
box = alt.Chart(df_gb).mark_boxplot().encode(
    y=alt.Y('case_duration_days:Q'),
    x=alt.X('muni:N'),
    color='muni:N',
)

# viz using histogram
hist = alt.Chart(df_gb).mark_bar(
).encode(
    x=alt.X('case_duration_days:Q').bin(maxbins=20),
    y=alt.Y('count()').stack(None),
    color='muni:N',
    column='muni:N'
)

# viz using KDE
hist_den = alt.Chart(df_gb).mark_area().transform_density(
    'case_duration_days',
    groupby=['muni'],
    as_=['case_duration_days', 'density'],
).encode(
    x=alt.X('case_duration_days:Q'),
    y=alt.Y('density:Q').stack(None),
    color='muni:N'
).properties(
    width=1000
)

alt.vconcat(box, hist, hist_den)

KruskalResult(statistic=np.float64(222.95148260547666), pvalue=np.float64(4.342697216408182e-47))


alt.VConcatChart(...)

### Result (related to Kap analysis only)

- At least two of the municipalities are different

In [53]:
# Identify the pair(s) of municipalities with significant differences in case duration
# Pairwise Wilcoxon Rank Sum Test
# Condition 1: two samples being random and independent
# Condition 2: two distributions are continuous
# H0: distributions are identical
# Ha: one distribution shifted either to the left or to the right of the other distribution
from itertools import combinations
from scipy.stats import ranksums

for x, y in combinations(sorted(df_gb['muni'].unique()), r=2):
    print('Wilcoxon Rank Sum Test for {} and {}'.format(x, y))
    arr_x = df_gb.loc[df_gb['muni'] == x, 'case_duration_days']
    arr_y = df_gb.loc[df_gb['muni'] == y, 'case_duration_days']
    print('\t{}'.format(ranksums(arr_x, arr_y)))

Wilcoxon Rank Sum Test for muni-1 and muni-2
	RanksumsResult(statistic=np.float64(4.636855220367986), pvalue=np.float64(3.5375000788314336e-06))
Wilcoxon Rank Sum Test for muni-1 and muni-3
	RanksumsResult(statistic=np.float64(10.861453653042318), pvalue=np.float64(1.7592615869755127e-27))
Wilcoxon Rank Sum Test for muni-1 and muni-4
	RanksumsResult(statistic=np.float64(-1.0752810557628223), pvalue=np.float64(0.28224891514319694))
Wilcoxon Rank Sum Test for muni-1 and muni-5
	RanksumsResult(statistic=np.float64(1.2985424301501822), pvalue=np.float64(0.19410100597565683))
Wilcoxon Rank Sum Test for muni-2 and muni-3
	RanksumsResult(statistic=np.float64(8.181945424465779), pvalue=np.float64(2.7929755222065136e-16))
Wilcoxon Rank Sum Test for muni-2 and muni-4
	RanksumsResult(statistic=np.float64(-4.651504816812931), pvalue=np.float64(3.295215764865707e-06))
Wilcoxon Rank Sum Test for muni-2 and muni-5
	RanksumsResult(statistic=np.float64(-2.361326724995864), pvalue=np.float64(0.018209679

### Result (related to Kap analysis only)

- Municipalities 1-4-5 may be drawn from identical distributions, with p > 0.05 (cannot reject H0)
- Municipalities 2-1, 2-4 are not likely to be drawn from identical distributions, with p << 0.05 (reject H0); 2-5 is slightly more similar with p ~= 0.01
- Municipality 3 is not likely to be drawn from a distribution identical to any others

In [54]:
# Analyze if and how case performance differ in each municipality, considering case types

charts = []
for muni, data in df_gb.groupby('muni'):
    print(muni)
    if muni in ['muni-1', 'muni-2', 'muni-5']:
        continue
    print('{} Unique case types: {}'.format(data['case_type'].nunique(), data['case_type'].unique()))
    print('Average case duration (days): {}'.format(data['case_duration_days'].mean()))
    # print(data['case_duration_days'].describe())
    # muni_chart = alt.Chart(data).mark_circle(size=60).encode(
    #     x='num_events:Q',
    #     y='case_duration_days:Q',
    #     color='case_type:N',
    #     tooltip='case_type:N'
    # )
    muni_chart = alt.Chart(
        data, title=[
            '{}:'.format(muni),
            'Avg. case duration {:.1f} days'.format(data['case_duration_days'].mean()),
            '50% completed in {:.1f} days, 90% completed in {:.1f} days'.format(
                data['case_duration_days'].median(),
                data['case_duration_days'].quantile(.9)
            )
        ]
    )
    
    hist = muni_chart.mark_bar().encode(
        y=alt.Y('count()').title('Number of cases'),
        x=alt.X('case_duration_days:Q', bin=alt.Bin(extent=[0, 360], step=20)).title('Case duration (days)'),
        color=alt.Color(
            'case_type:N', 
            sort=[
                'CT.1 (Procedure resumed)', 'CT.2 (Decision irrevocable)', 'CT.3 (Case handled)', 'CT.4 (Application received)', 'CT.5 (Procedure terminated interim)', 'CT.6 (Decision made)', 
                'CT.7 (Draft decision made)', 'CT.8 (Procedure aborted)', 'CT.9 (Activity exempt from permit)',
                'CT.10 (Additional information requested)', 'CT.11 (Decision ready)', 'CT.12 (Forwarded to competent authority)', 
                'CT.13 (Permit irrevocable)', 'CT.14 (Objection filed)', 'CT.15 (Advice known)', 'CT.16 (Decision sent/Permit granted)', 
            ]
        ),
        tooltip=['case_type', 'count()']
    )
    rule_mean = muni_chart.mark_rule(color='blue').encode(
        x=alt.X('mean(case_duration_days):Q').title(''),
        size=alt.value(2)
    ) 
    rule_med = muni_chart.transform_quantile(
        'case_duration_days', 
        probs=[0.5],
        as_=['prob', 'value']
    ).mark_rule(color='red').encode(
        x=alt.X('value:Q').title(''),
        size=alt.value(2)
    )
    rule_hi10 = muni_chart.transform_quantile(
        'case_duration_days', 
        probs=[0.9],
        as_=['prob', 'value']
    ).mark_rule(color='red', strokeDash=[5, 5]).encode(
        x=alt.X('value:Q').title(''),
        size=alt.value(2)
    )
    
    hist = hist + rule_mean + rule_med + rule_hi10
    
    hist2 = alt.Chart(
        data, 
        title='Pct. of different types of case in {}'.format(muni)
    ).transform_joinaggregate(
        total='count()'
    ).transform_calculate(
        cnt='1',
        pct='1 / datum.total'
    ).mark_bar().encode(
        alt.Y('case_type:N').title('Case type'),
        alt.X('sum(pct):Q', axis=alt.Axis(format='%'), scale=alt.Scale(domain=[0, 1.0])).title('Percentage out of all cases'),
        # color=alt.Color(
        #     'case_type:N', 
        #     sort=[
        #         'CT.1 (Procedure resumed)', 'CT.2 (Decision irrevocable)', 'CT.3 (Case handled)', 'CT.4 (Application received)', 'CT.5 (Procedure terminated interim)', 'CT.6 (Decision made)', 
        #         'CT.7 (Draft decision made)', 'CT.8 (Procedure aborted)', 'CT.9 (Activity exempt from permit)',
        #         'CT.10 (Additional information requested)', 'CT.11 (Decision ready)', 'CT.12 (Forwarded to competent authority)', 
        #         'CT.13 (Permit irrevocable)', 'CT.14 (Objection filed)', 'CT.15 (Advice known)', 'CT.16 (Decision sent/Permit granted)', 
        #     ]
        # )
    )
    hist2 = alt.layer(
        hist2,
        hist2.mark_text(
            align='left',
            baseline='middle',
            dx=10
        ).encode(
            text=alt.Text('sum(cnt):Q')
        )
    )
    
    bar = alt.Chart(
        data, 
        title='Avg. case duration for each case type in {}'.format(muni)
    ).mark_bar().encode(
        alt.Y('case_type:N').title('Case type'),
        alt.X('mean(case_duration_days):Q', scale=alt.Scale(domain=[0, 200])).title('Case duration (days)'),
        color=alt.Color(
            'case_type:N', 
            sort=[
                'CT.1 (Procedure resumed)', 'CT.2 (Decision irrevocable)', 'CT.3 (Case handled)', 'CT.4 (Application received)', 'CT.5 (Procedure terminated interim)', 'CT.6 (Decision made)', 
                'CT.7 (Draft decision made)', 'CT.8 (Procedure aborted)', 'CT.9 (Activity exempt from permit)',
                'CT.10 (Additional information requested)', 'CT.11 (Decision ready)', 'CT.12 (Forwarded to competent authority)', 
                'CT.13 (Permit irrevocable)', 'CT.14 (Objection filed)', 'CT.15 (Advice known)', 'CT.16 (Decision sent/Permit granted)', 
            ]
        ),
    )
    
    bar2 = alt.Chart(
        data, 
        title='Total (sum of) case duration for each case type in {}'.format(muni)
    ).mark_bar().encode(
        alt.Y('case_type:N').title('Case type'),
        alt.X('sum(case_duration_days):Q').title('Case duration (days)'),
        color=alt.Color(
            'case_type:N', 
            sort=[
                'CT.1 (Procedure resumed)', 'CT.2 (Decision irrevocable)', 'CT.3 (Case handled)', 'CT.4 (Application received)', 'CT.5 (Procedure terminated interim)', 'CT.6 (Decision made)', 
                'CT.7 (Draft decision made)', 'CT.8 (Procedure aborted)', 'CT.9 (Activity exempt from permit)',
                'CT.10 (Additional information requested)', 'CT.11 (Decision ready)', 'CT.12 (Forwarded to competent authority)', 
                'CT.13 (Permit irrevocable)', 'CT.14 (Objection filed)', 'CT.15 (Advice known)', 'CT.16 (Decision sent/Permit granted)', 
            ]
        ),
    )
        
    charts.append(alt.hconcat(bar, hist2))
    
    # freq_case_types = [ct for ct in data['case_type'].unique() if data.loc[data['case_type'] == ct, 'case:concept:name'].nunique() > 5]
    # print('{} Frequent case types (with more than 5 cases):'.format(len(freq_case_types)))
    # print(freq_case_types)
    # case_dur_cts = [data.loc[data['case_type'] == ct, 'case_duration_days'] for ct in freq_case_types]
    # # H (i.e., Chi-Square) and p-value
    # print('Performing Kruskal Wallis H Test:')
    # print(kruskal(*case_dur_cts))

alt.vconcat(*charts)

muni-1
muni-2
muni-3
9 Unique case types: ['CT.7 (Draft decision made)' 'CT.15 (Advice known)'
 'CT.4 (Application received)' 'CT.16 (Decision sent/Permit granted)'
 'CT.6 (Decision made)' 'CT.2 (Decision irrevocable)'
 'CT.5 (Procedure terminated interim)' 'CT.8 (Procedure aborted)'
 'CT.10 (Additional information requested)']
Average case duration (days): 24.058492366412217
muni-4
8 Unique case types: ['CT.6 (Decision made)' 'CT.8 (Procedure aborted)'
 'CT.13 (Permit irrevocable)' 'CT.2 (Decision irrevocable)'
 'CT.14 (Objection filed)' 'CT.9 (Activity exempt from permit)'
 'CT.12 (Forwarded to competent authority)'
 'CT.16 (Decision sent/Permit granted)']
Average case duration (days): 90.47126623376624
muni-5


alt.VConcatChart(...)

In [55]:
# Analyze if and how case performance differ in each municipality, considering case types
charts = []
for muni, data in df_gb.groupby('muni'):
    print(muni)
    if muni in ['muni-1', 'muni-4', 'muni-5']:
        data['muni_type'] = 'Lower-performing municipalities'
    else:
        data['muni_type'] = 'Higher-performing '

    data['case_type'] = data['case_type'].apply(lambda s: s.split('(')[0][:-1])
    print(data.groupby('case_type').size() / data['case:concept:name'].nunique())
    
    hist2 = alt.Chart(
        data, 
        title='{}'.format(muni)
    ).transform_joinaggregate(
        total='count()'
    ).transform_calculate(
        cnt='1',
        pct='1 / datum.total'
    ).mark_bar().encode(
        y=alt.Y(
            'case_type:N', 
            sort=['CT.{}'.format(x) for x in range(1, 17)],
            axis=alt.Axis(labels=(muni == 'muni-1'))
        ).title('Case type' if muni == 'muni-1' else ''),
        x=alt.X('sum(pct):Q', axis=alt.Axis(format='%'), scale=alt.Scale(domain=[0, 1.0])).title('Percentage of cases' if muni == 'muni-3' else ''),
        color=alt.Color('muni_type').legend(orient='top')
    ).properties(
        height=150, width=175
    )
  
    charts.append(hist2)

alt.hconcat(*charts).resolve_scale(y='shared')

muni-1
case_type
CT.1     0.011111
CT.15    0.088889
CT.16    0.288889
CT.2     0.022222
CT.3     0.477778
CT.4     0.044444
CT.7     0.011111
CT.8     0.055556
dtype: float64
muni-2
case_type
CT.11    0.010526
CT.15    0.010526
CT.16    0.052632
CT.3     0.905263
CT.4     0.010526
CT.9     0.010526
dtype: float64
muni-3
case_type
CT.10    0.007634
CT.15    0.122137
CT.16    0.767176
CT.2     0.007634
CT.4     0.026718
CT.5     0.003817
CT.6     0.003817
CT.7     0.057252
CT.8     0.003817
dtype: float64
muni-4
case_type
CT.12    0.006494
CT.13    0.012987
CT.14    0.006494
CT.16    0.084416
CT.2     0.493506
CT.6     0.298701
CT.8     0.090909
CT.9     0.006494
dtype: float64
muni-5
case_type
CT.13    0.318182
CT.14    0.015152
CT.15    0.015152
CT.16    0.090909
CT.2     0.106061
CT.3     0.393939
CT.4     0.015152
CT.5     0.015152
CT.8     0.015152
CT.9     0.015152
dtype: float64


alt.HConcatChart(...)

In [56]:
charts = []
for muni, data in df_gb.groupby('muni'):
    print(muni)
    if muni in ['muni-1', 'muni-2', 'muni-5']:
        continue

    # SOLUTION 1: Use scatter plot
    
    scatter = alt.Chart(
        data, 
        title='{}'.format(muni)
    ).transform_joinaggregate(
        total='count()'
    ).transform_calculate(
        pct='1 / datum.total'
    ).encode(
        alt.X('sum(pct):Q', axis=alt.Axis(format='%'), scale=alt.Scale(domain=[0, 1.0])).title('Percentage out of all cases'),
        alt.Y('mean(case_duration_days):Q', scale=alt.Scale(domain=[0, 200])).title('Average case duration (days)'),
    )      

    scatter = alt.layer(
        scatter.mark_circle(size=120).encode(
            color=alt.Color(
                'case_type:N',
                sort=[
                    'CT.1 (Procedure resumed)', 'CT.2 (Decision irrevocable)', 'CT.3 (Case handled)', 'CT.4 (Application received)', 'CT.5 (Procedure terminated interim)', 'CT.6 (Decision made)', 
                    'CT.7 (Draft decision made)', 'CT.8 (Procedure aborted)', 'CT.9 (Activity exempt from permit)',
                    'CT.10 (Additional information requested)', 'CT.11 (Decision ready)', 'CT.12 (Forwarded to competent authority)', 
                    'CT.13 (Permit irrevocable)', 'CT.14 (Objection filed)', 'CT.15 (Advice known)', 'CT.16 (Decision sent/Permit granted)', 
                ]
            ).title('Case type'),
            # size=alt.Size('mean(case_duration_days):Q').legend(None)
        ),
        scatter.mark_text(
            align='left',
            baseline='middle',
            dx=15
        ).encode(
            text='case_type'
        )
    )

    # SOLUTION 2: Use grouped bar chart with dual Y-axes (requires post-editing to remove dummy chart)
    
    # export raw data
    num_cases = data['case:concept:name'].nunique()
    data_out = data.groupby('case_type').agg(
        pct=pd.NamedAgg(column='case:concept:name', aggfunc='size'),
        mean_case_duration=pd.NamedAgg(column='case_duration_days', aggfunc='mean')
    )
    data_out['pct'] = data_out['pct'] / num_cases
    # data_out.to_csv('{}_details.csv'.format(muni))


    data_out = data_out.reset_index()
    data_out['case_type'] = data_out['case_type'].apply(lambda s: s.split('(')[0][:-1])
    data_out['dummy_mcd'] = data_out['mean_case_duration'] / 200
    data_out = pd.melt(data_out, id_vars=['case_type'], value_vars=['pct', 'dummy_mcd', 'mean_case_duration'])
    missing_ct = []
    for ctn in [2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16]:
        ct = 'CT.{}'.format(ctn)
        if len(data_out[data_out['case_type'] == ct]) == 0:
            missing_ct.append(ct)
    for ct in missing_ct:
        data_out = pd.concat([
            data_out,
            pd.DataFrame({
                'case_type': [ct, ct, ct],
                'variable': ['pct', 'dummy_mcd', 'mean_case_duration'],
                'value': [0, 0, 0]
            })
        ])
    print(data_out)

    # cl: dummy chart just to get the axis
    cl = alt.Chart(title=muni, data=data_out[data_out['variable'] == 'mean_case_duration']).mark_bar().encode(
        color=alt.value('grey'),
        column=alt.Column('case_type:N', spacing=0).title(''),
        x=alt.X('variable:N', axis=alt.Axis(title=None, labels=False)),
        y=alt.Y('value:Q', scale=alt.Scale(domain=[0, 200])).title('Average case duration (days)')
    ).properties(height=150, width=30)
    cr = alt.Chart(title=muni, data=data_out[data_out['variable'].isin(['dummy_mcd', 'pct'])]).mark_bar().encode(
        column=alt.Column(
            'case_type:N', 
            sort=['CT.{}'.format(x) for x in range(1, 17)],
        spacing=0).title(''),
        color=alt.Color('variable:N', sort=['dummy_mcd', 'pct']),
        x=alt.X('variable:N', axis=alt.Axis(title=None, labels=False)),
        y=alt.Y('value:Q', axis=alt.Axis(format='%', orient='right'), scale=alt.Scale(domain=[0, 1.0])).title('Percentage out of all cases')
    ).properties(height=150, width=30)
    
    charts.append(alt.hconcat(cl, cr))

alt.hconcat(*charts)

muni-1
muni-2
muni-3
   case_type            variable      value
0      CT.10                 pct   0.007634
1      CT.15                 pct   0.122137
2      CT.16                 pct   0.767176
3       CT.2                 pct   0.007634
4       CT.4                 pct   0.026718
5       CT.5                 pct   0.003817
6       CT.6                 pct   0.003817
7       CT.7                 pct   0.057252
8       CT.8                 pct   0.003817
9      CT.10           dummy_mcd   0.074896
10     CT.15           dummy_mcd   0.147718
11     CT.16           dummy_mcd   0.114714
12      CT.2           dummy_mcd   0.057500
13      CT.4           dummy_mcd   0.056863
14      CT.5           dummy_mcd   0.027958
15      CT.6           dummy_mcd   0.328177
16      CT.7           dummy_mcd   0.174210
17      CT.8           dummy_mcd   0.100000
18     CT.10  mean_case_duration  14.979167
19     CT.15  mean_case_duration  29.543576
20     CT.16  mean_case_duration  22.942817
21      CT.

alt.HConcatChart(...)

### Result (related to Kap analysis only)

- Municipalities 2 and 3 clearly had shorter case duration time on the 'Kap' cases
- Municipalities 1-4-5 had longer duration time and are closer to one another

## Resource-oriented analysis

In [57]:
# Visualize group distribution w.r.t. different types of cases (adapted from "member assignment")
# Can we blame specific resources for slow cases?

charts = []
for muni, events in df.groupby('municipality'):
    if muni in ['muni-1', 'muni-2', 'muni-5']:
        continue
    data = []
    num_cases = events['case:concept:name'].nunique()
    for case, trace in events.groupby('case:concept:name'):
        resources = set(trace['org:resource'].unique())
        ct = trace.iloc[0]['case_type']
        dur = trace.iloc[0]['case_duration_days']
        for r in resources:
            data.append({'resource': r, 'case_type': ct, '#cases': 1, 'mean_dur_days': dur})
    data = pd.DataFrame(data)
    data = data.groupby(['resource', 'case_type']).agg({'#cases': 'sum', 'mean_dur_days': 'mean'})
    data['pct_cases'] = data['#cases'] / num_cases
    heatmap_perf = alt.Chart(data=data, title=f'{muni}').mark_rect().encode(
        x='resource:O',
        y='case_type:O',
        color=alt.Color('mean_dur_days:Q', scale=alt.Scale(scheme='lightgreyred')),
        tooltip=['mean_dur_days', '#cases']
    )
    heatmap_assi = alt.Chart(data=data, title=f'{muni}')
    heatmap_assi = alt.layer(
        heatmap_assi.mark_rect().encode(
            x=alt.X('resource:O').title('Resource'),
            y=alt.Y(
                'case_type:N',
                sort=[
                    'CT.1 (Procedure resumed)', 'CT.2 (Decision irrevocable)', 'CT.3 (Case handled)', 'CT.4 (Application received)', 'CT.5 (Procedure terminated interim)', 'CT.6 (Decision made)', 
                    'CT.7 (Draft decision made)', 'CT.8 (Procedure aborted)', 'CT.9 (Activity exempt from permit)',
                    'CT.10 (Additional information requested)', 'CT.11 (Decision ready)', 'CT.12 (Forwarded to competent authority)', 
                    'CT.13 (Permit irrevocable)', 'CT.14 (Objection filed)', 'CT.15 (Advice known)', 'CT.16 (Decision sent/Permit granted)', 
                ]
            ).title(['Case type', '(case endpoints)']),
            color=alt.Color('#cases:Q', scale=alt.Scale(scheme='lightgreyred')).title('Num. of cases'),
            tooltip=['mean_dur_days', '#cases']
        ),
        heatmap_assi.mark_text(baseline='middle').encode(
            x='resource:O',
            y=alt.Y(
                'case_type:N',
                sort=[
                    'CT.1 (Procedure resumed)', 'CT.2 (Decision irrevocable)', 'CT.3 (Case handled)', 'CT.4 (Application received)', 'CT.5 (Procedure terminated interim)', 'CT.6 (Decision made)', 
                    'CT.7 (Draft decision made)', 'CT.8 (Procedure aborted)', 'CT.9 (Activity exempt from permit)',
                    'CT.10 (Additional information requested)', 'CT.11 (Decision ready)', 'CT.12 (Forwarded to competent authority)', 
                    'CT.13 (Permit irrevocable)', 'CT.14 (Objection filed)', 'CT.15 (Advice known)', 'CT.16 (Decision sent/Permit granted)', 
                ]
            ),
            text=alt.Text('#cases:Q')
        )
    )
    charts.append(
        heatmap_assi
        # alt.hconcat(heatmap_assi).resolve_scale(color='independent')
    )
alt.hconcat(*charts).resolve_scale(y='shared', color='independent')

KeyError: "['case_type', 'resource'] not in index"

alt.HConcatChart(...)

In [58]:
# Visualize group distribution w.r.t. different execution contexts ("member assignment", which is event-based)
# Consider all cases or the major / most time-consuming cases
# Exclude the first event (no duration estimate available)

df = df[df['activity_duration_days'].notna()]

charts = []
sel_case_types = {
    'muni-1': 'CT.3 (Case handled)',
    
    'muni-2': 'CT.3 (Case handled)',
    
    'muni-3': 'CT.16 (Decision sent/Permit granted)',
    # 'muni-3': 'CT.6 (Decision made)',
    
    'muni-4': 'CT.2 (Decision irrevocable)',
    # 'muni-4': 'CT.6 (Decision made)',
    
    'muni-5': 'CT.3 (Case handled)',
    # 'muni-5': 'CT.13 (Permit irrevocable)',
}

for muni, events in df.groupby('municipality'):
    data = []

    if muni in ['muni-1', 'muni-2', 'muni-5']:
        continue
        
    ct_events = events.rename(
        columns={'org:resource': 'resource'}
    )
    # ct_events = events[events['case_type'] == sel_case_types[muni]]
    print(ct_events['case_type'].nunique())
    print('#events = {}'.format(len(events)))
    
    # ct_events_gb = ct_events.groupby(
    #     by=['resource', 'case_type', 'activity_type', 'time_type']
    # ).agg(
    #     count=pd.NamedAgg(column='resource', aggfunc=len),
    #     mean_activity_duration_days=pd.NamedAgg(column='activity_duration_days', aggfunc='mean')
    # ).reset_index()
    cols = ['resource', 'case_type', 'activity_type', 'time_type', 'activity_duration_days']

    # heatmap_act_time = alt.Chart(
    #     data=ct_events[cols], title=f'{muni}').mark_rect().encode(
    #     x=alt.X('activity_type:O').title('Activity type'),
    #     y=alt.Y('time_type:O').title('Time type'),
    #     color=alt.Color('count()', scale=alt.Scale(domain=[0, len(ct_events)], scheme='lightgreyred')),
    #     tooltip=['count()']
    # )
    # heatmap_act_time.save('{}-act_time.svg'.format(muni))
    heatmap_res_act_count = alt.Chart(data=ct_events[cols], title=f'{muni}')
    heatmap_res_act_count = alt.layer(
        heatmap_res_act_count.mark_rect().encode(
            x=alt.X('resource:O').title('Resource'),
            y=alt.Y('activity_type:O').title(['Activity type', '(subprocess phases)']),
            color=alt.Color('count()', scale=alt.Scale(domain=[0, len(ct_events)], scheme='lightgreyred')).title('Num. of activities'),
            tooltip=['count()']
        ),
        heatmap_res_act_count.mark_text(baseline='middle', size=7.5).encode(
            x=alt.X('resource:O'),
            y=alt.Y('activity_type:O'),
            text=alt.Text('count()')
        )
    )
    # heatmap_res_act_count.save('{}-res_act_count.svg'.format(muni))
    
    heatmap_res_act_perf = alt.Chart(
        data=ct_events[cols], title=f'{muni}').mark_rect().encode(
        x=alt.X('resource:O').title('Resource'),
        y=alt.Y('activity_type:O').title(['Activity type', '(subprocess phases)']),
        color=alt.Color('mean(activity_duration_days):Q', scale=alt.Scale(scheme='lightgreyred')).title('Avg. activity duration (days)'),
        tooltip=['mean(activity_duration_days)']
    )
    # heatmap_res_act_perf.save('{}-res_act_perf.svg'.format(muni))
    
    heatmap_res_time_count = alt.Chart(data=ct_events[cols], title=f'{muni}')
    heatmap_res_time_count = alt.layer(
        heatmap_res_time_count.mark_rect().encode(
            x=alt.X('resource:O').title('Resource'),
            y=alt.Y('time_type:O').title(['Time type', '(weekdays)']),
            color=alt.Color('count()', scale=alt.Scale(domain=[0, len(ct_events)], scheme='lightgreyred')).title('Num. of activities'),
            tooltip=['count()']
        ),
        heatmap_res_time_count.mark_text(baseline='middle', size=7.5).encode(
            x=alt.X('resource:O'),
            y=alt.Y('time_type:O'),
            text=alt.Text('count()')
        )
    )
    # heatmap_res_time_count.save('{}-res_time_count.pdf'.format(muni))
    
    heatmap_res_time_perf = alt.Chart(
        data=ct_events[cols], title=f'{muni}').mark_rect().encode(
        x=alt.X('resource:O').title('Resource'),
        y=alt.Y('time_type:O').title(['Time type', '(weekdays)']),
        color=alt.Color('mean(activity_duration_days):Q', scale=alt.Scale(scheme='lightgreyred')).title('Avg. activity duration (days)'),
        tooltip=['mean(activity_duration_days)']
    )
    # heatmap_res_time_perf.save('{}-res_time_perf.pdf'.format(muni))
    
    charts.append(
        alt.hconcat(
            alt.vconcat(heatmap_res_act_count, heatmap_res_act_perf).resolve_scale(color='independent'),
            alt.vconcat(heatmap_res_time_count, heatmap_res_time_perf).resolve_scale(color='independent')
        ).resolve_scale(color='independent')
    )
alt.vconcat(*charts).resolve_scale(color='independent')

9
#events = 8819
8
#events = 5445


alt.VConcatChart(...)

In [59]:
# Visualize group distribution w.r.t. different execution contexts ("member assignment", which is event-based)
# Considering the slowest cases

charts = []
sel_case_thresholds = {
    'muni-1': 112.5, 'muni-2': 98.9, 'muni-3': 44.8, 'muni-4': 162.5, 'muni-5': 113.9
}

for muni, events in df.groupby('municipality'):
    data = []
    sel_cases = []
    for case, trace in events.groupby('case:concept:name'):
        if trace.iloc[0]['case_duration_days'] >= sel_case_thresholds[muni]:
            sel_cases.append(case)
    ct_events = events[events['case:concept:name'].isin(sel_cases)]
    ct_events_gb = ct_events.groupby(by=['municipality', 'org:resource', 'case_type', 'activity_type', 'time_type']).size().reset_index().rename(
        columns={
            'org:resource': 'resource',
            0: 'count'
        }
    )
    charts.append(
        alt.hconcat(
            alt.Chart(data=ct_events_gb, title=f'{muni}').mark_rect().encode(
                x='activity_type:O',
                y='time_type:O',
                color=alt.Color('sum(count):Q', scale=alt.Scale(scheme='lightgreyred')),
                tooltip=['sum(count)']
            ),
            alt.Chart(data=ct_events_gb, title=f'{muni}').mark_rect().encode(
                x='resource:O',
                y='activity_type:O',
                color=alt.Color('sum(count):Q', scale=alt.Scale(scheme='lightgreyred')),
                tooltip=['sum(count)']
            ),
            alt.Chart(data=ct_events_gb, title=f'{muni}').mark_rect().encode(
                x='resource:O',
                y='time_type:O',
                color=alt.Color('sum(count):Q', scale=alt.Scale(scheme='lightgreyred')),
                tooltip=['sum(count)']
            ),
        )
    )
alt.vconcat(*charts).resolve_scale(color='independent')

alt.VConcatChart(...)

In [60]:
# Create a bipartite graph for each municipality, where
# nodes represent resources and case types
# links represent "working together" (between resources) and "participating in" (between resource and case types)

if False:
    for muni, data in df.groupby('municipality'):
        # TODO: define CT node size by mean case duration time of that CT
        # TODO: define CT node color by their colors in the histogram above
        
        ct_mean_case_dur = {}
        for ct, events in data.groupby('case_type'):
            case_dur = events.drop_duplicates(subset=['case:concept:name'])['case_duration_days'].mean()
            ct_mean_case_dur[ct] = case_dur
        
        g = nx.Graph()
        for case, events in data.groupby('case:concept:name'):
            ct = events.iloc[0]['case_type']
            if not g.has_node(ct):
                g.add_node(ct, _class='Case Type', label=ct, size=ct_mean_case_dur[ct])
            resources = set(events['org:resource'].unique())
            for r in resources:
                if not g.has_node(r):
                    g.add_node(r, _class='org:resource')
                if not g.has_edge(r, ct):
                    g.add_edge(r, ct, weight=1)
                else:
                    g[r][ct]['weight'] += 1

            for rx, ry in combinations(resources, r=2):
                if not g.has_edge(rx, ry):
                    g.add_edge(rx, ry, weight=1)
                else:
                    g[rx][ry]['weight'] += 1
        nx.write_graphml(g, '{}.graphml'.format(muni))